In [1]:
import numpy as np
import pandas as pd
import heapq

In [3]:
origin_data = pd.read_excel('结果表4.xlsx',header=0)#读取数据文件
origin_data = origin_data[origin_data['分拣中心']=='SC60']
dates = origin_data['日期'].unique() #获取每天日期的列表形式

In [ ]:
origin_data

,分拣中心,日期,小时,最终货量
35280,SC60,2023-12-01,0,4180.619895
35281,SC60,2023-12-01,1,1550.710722
35282,SC60,2023-12-01,2,4824.562273
35283,SC60,2023-12-01,3,4753.551695
35284,SC60,2023-12-01,4,3554.103336
...,...,...,...,...
35995,SC60,2023-12-30,19,3537.444216
35996,SC60,2023-12-30,20,3806.576898
35997,SC60,2023-12-30,21,4231.566364
35998,SC60,2023-12-30,22,3297.940970


In [90]:
#定义时间段类，类中包含了该时间段的正式工、临时工人数，待分拣包裹数等信息
class Time_weight:
    sum_x_workers=0#约束条件为小于160
    sum_y_workers=0
    def __init__(self,begin,end , a=0, b=0):
        self.a = a#时间段里，包裹数超过20的小时数量
        self.b = b #时间段里，包裹数大于0的小时数量
        self.sum = 0 #时间段里包裹数总和
        self.x_worker=0 #该时间段正式工人数,约束条件为<160
        self.y_worker=0 #该时间段里临时工人数
        self.begin=begin#时间段开始时间
        self.end=end #时间段结束时间
        self.xlist=[] #正式工列表，存储在该时间段工作的正式工人编号
    
    #重载>号运算，更改优先级
    def __gt__(self, other):
        #时间段里，包裹数超过20的小时数量最多的优先级最好，其次比较时间段里包裹总数，最后比较包裹数大于0的小时数量
        if self.a > other.a:
            return True
        elif self.a == other.a:
            if self.sum > other.sum:
                return True
            elif self.sum == other.sum:
                return self.b > other.b
        return False

    def __repr__(self):
        return f"时间段{self.begin}至{self.end}的正式工人数为{self.x_worker},临时工人数为{self.y_worker}"
    
    #每加入新的工人，更新a，b、sum的数值
    def update(self,tasks_hourly):
        self.a=0
        self.b=0
        self.sum=0
        # for task in tasks_hourly[self.begin:self.end]:
        for i in range(self.begin,self.end):
            if tasks_hourly[i]<0:
                tasks_hourly[i]=0
            self.sum+=tasks_hourly[i]
            if tasks_hourly[i] > 20:
                self.a+=1
            elif tasks_hourly[i]>0:
                self.b+=1
        return tasks_hourly
    #根据时间段内剩余的包裹，按照优先级增加工人，并更新时间段内的包裹
    def add_worker(self,tasks_hourly,worker):
        if self.__class__.sum_x_workers<165:
            self.x_worker+=1
            self.__class__.sum_x_workers+=1
            for i in range(self.begin,self.end):
                tasks_hourly[i]-=25
            self.xlist.append(worker.name)
            worker.shangban()
        else:
            self.y_worker+=1
            self.__class__.sum_y_workers+=1
            for i in range(self.begin,self.end):
                tasks_hourly[i]-=20
            # worker.continuous=0
        return tasks_hourly,worker  # 返回修改后的 tasks_hourly

In [61]:
#定义正式工类，类中包含正式工工作时间的信息，连续工作时间等信息
class Formal_worker:
    #定义初始化函数
    def __init__(self,name):
        self.name=name #正式工编号
        self.continuous = 0 #工人连续工作天数，约束<=7
        self.sum = 0 #工作总天数，约束<=25
        self.work_time=[] #工作的具体天数和时间段
        self.is_yesterday=False#代表昨天有没有出勤，如果0表示昨天没有出勤，1表示昨天出勤了，在新的一天将此数诚意连续工作天数后归0
        
    #重载>号运算，更改优先级
    def __gt__(self, other):
        #时间段里，包裹数超过20的小时数量最多的优先级最好，其次比较时间段里包裹总数，最后比较包裹数大于0的小时数量
        if self.sum > other.sum:
            return True
        elif self.sum == other.sum:
            if self.continuous > other.continuous:
                return True
            else:
                return False
        return False
     #重载小于号运算
    def __lt__(self, other):
        # 首先比较工作总天数
        if self.sum < other.sum:
            return True
        elif self.sum == other.sum:
            # 如果工作总天数相同，则比较连续工作天数
            if self.continuous < other.continuous:
                return True
            elif self.continuous == other.continuous:
                if self.is_yesterday and other.is_yesterday==False:
                    return True
                else :
                    return False
        # 如果工作总天数不同，则直接返回False
        return False
    def __repr__(self):
        return f"编号{self.name}的正式工人，工作总天数为{self.sum},连续工作天数为{self.continuous}"
    def isok(self):
        return self.sum<=25 and self.continuous<=7
    def shangban(self):
        self.continuous+=1
        self.sum+=1
        self.is_yesterday=True

#按日期遍历每天的包裹任务并根据工人与包裹情况进行排班
def greedy_select(tasks_hourly,min_heap):
    Time_weight.sum_x_workers=0
    Time_weight.sum_y_workers=0
    #申明时间段对象列表，分别代表六个时间端
    time_list = [Time_weight(0,8),Time_weight(5,13),Time_weight(8,16),Time_weight(12,20),Time_weight(14,22),Time_weight(16,24)]
     #首先根据24小时的任务量更新时间段对象
    for i in range(6):
        time_list[i].update(tasks_hourly)
    #当时间段内还有包裹没有完成分拣时，一直循环
    while max(tasks_hourly)>0:
        #如果最小的正式工满足约束条件
        if min_heap[0].isok():
            smallest_worker=heapq.heappop(min_heap)
            # for worker in min_heap:
            #     print(worker)
            tasks_hourly,smallest_worker=max(time_list).add_worker(tasks_hourly,smallest_worker)
            heapq.heappush(min_heap,smallest_worker)

        else:
            print('不应该啊！')
            for worker in min_heap:
                print(worker)
            break
            
        for i in range(6):
            time_list[i].update(tasks_hourly)
        # print(tasks_hourly)
    print(Time_weight.sum_x_workers)
    print(Time_weight.sum_y_workers)
    for i in range(6):
        print(time_list[i])

In [131]:
#按日期遍历每天的包裹任务并根据工人与包裹情况进行排班
def greedy_select(date,tasks_hourly,workers):
    tasks=tasks_hourly.copy()
    tasks.astype(float)
    #新的一天初始化当天员工数量
    Time_weight.sum_x_workers=0
    Time_weight.sum_y_workers=0
    for w in workers:
        if w.is_yesterday==False:
            w.continuous=0
        else:
            w.is_yesterday=False
    #申明时间段对象列表，分别代表六个时间端
    time_list = [Time_weight(0,8),Time_weight(5,13),Time_weight(8,16),Time_weight(12,20),Time_weight(14,22),Time_weight(16,24)]
     #首先根据24小时的任务量更新时间段对象
    for i in range(6):
        time_list[i].update(tasks_hourly)
    #当时间段内还有包裹没有完成分拣时，一直循环
    while max(tasks_hourly)>0:
        #如果最小的正式工满足约束条件
        # smallest_worker=min(workers)
        index,smallest_worker = min(enumerate(workers),key=lambda x:(x[1],x[0]))  
        if smallest_worker.isok():
            index_time,max_time= max(enumerate(time_list),key=lambda x:(x[1],x[0]))
            # print(max_time)
            tasks_hourly,workers[index]=time_list[index_time].add_worker(tasks_hourly,workers[index])
        else:
            print(smallest_worker)
            print('不应该啊！')
            for worker in min_heap:
                print(worker)
            break
        for i in range(6):
            tasks_hourly=time_list[i].update(tasks_hourly)

        # 定义班次时间
    shifts ={
        1: '00:00-08:00',
        2: '05:00-13:00',
        3: '08:00-16:00',
        4: '12:00-20:00',
        5: '14:00-22:00',
        6: '16:00-24:00'
    }
    result_list = []
    # for i in range(1,7):
    #     for j in time_list[i-1].xlist:
    #         result_dict = {}
    #         result_dict['分拣中心'] = 'SC60'
    #         result_dict['日期'] = date
    #         result_dict['班次'] = shifts[i]
    #         result_dict['出勤员工'] = f"正式工{j}"
    #         result_list.append(result_dict)
    #     for j in range(time_list[i-1].y_worker):
    #         result_dict = {}
    #         result_dict['分拣中心'] = 'SC60'
    #         result_dict['日期'] = date
    #         result_dict['班次'] = shifts[i]
    #         result_dict['出勤员工'] = f"临时工{j}"
    #         result_list.append(result_dict)
    #计算总人天数与每天的实际小时人效方差
    x=[time.x_worker for time in time_list]
    y=[time.y_worker for time in time_list]
    k=[
    [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]
]
    fangcha=[]
    for i in range(len(tasks)):
        s=0
        if tasks[i] != 0:
            for j in range(6):
                s+=k[j][i]*(x[j]+y[j])
            fangcha.append(tasks[i]/s)
    # print(fangcha)
    # print(round(np.var(fangcha),3))
    result_dict = {}
    result_dict['分拣中心'] = 'SC60'
    result_dict['日期'] = date
    result_dict['正式工人数'] = Time_weight.sum_x_workers
    result_dict['临时工人数'] = Time_weight.sum_y_workers
    result_dict['总人天数'] = Time_weight.sum_x_workers+Time_weight.sum_y_workers
    result_dict['实际小时人效方差']=round(np.var(fangcha),3)
    result_list.append(result_dict)
    return workers,result_list
    # for i in range(6):
    #     print(time_list[i])


In [132]:
# 调用函数并汇总结果
all_results = []
# 创建200个Formal_worker对象
workers = [Formal_worker(name=i) for i in range(1, 201)]
for date in dates:
    print(date)
    tasks = origin_data[origin_data['日期']==date]['最终货量']
    # print(tasks.values)
    tasks = tasks.values 
    tasks=tasks.astype(int)
    # print(tasks)
    workers,results = greedy_select(date,tasks,workers)
    if results:
        all_results.extend(results)
    else:
        print(date)

2023-12-01 00:00:00
2023-12-02 00:00:00
2023-12-03 00:00:00
2023-12-04 00:00:00
2023-12-05 00:00:00
2023-12-06 00:00:00
2023-12-07 00:00:00
2023-12-08 00:00:00
2023-12-09 00:00:00
2023-12-10 00:00:00
2023-12-11 00:00:00
2023-12-12 00:00:00
2023-12-13 00:00:00
2023-12-14 00:00:00
2023-12-15 00:00:00
2023-12-16 00:00:00
2023-12-17 00:00:00
2023-12-18 00:00:00
2023-12-19 00:00:00
2023-12-20 00:00:00
2023-12-21 00:00:00
2023-12-22 00:00:00
2023-12-23 00:00:00
2023-12-24 00:00:00
2023-12-25 00:00:00
2023-12-26 00:00:00
2023-12-27 00:00:00
2023-12-28 00:00:00
2023-12-29 00:00:00
2023-12-30 00:00:00


In [133]:
res = pd.DataFrame(all_results)
res.to_excel('问题4总人天数2.xlsx',index=False)

# 结果分析

In [93]:
chuqinlv = [worker.sum/30 for worker in workers]
print(chuqinlv)
print(np.var(chuqinlv))

[0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334,